# Attractors, stability, continuation

This lecture introduces the Attractors.jl library package of DynamicalSystems.jl,
which offers functionality for finding attractors of dynamical systems, studying global (non-local) stability, and performing continuation, among other things.

In [ ]:
import Pkg
Pkg.activate(dirname(@__DIR__))
Pkg.status(["DynamicalSystems", "Attractors", "CairoMakie", "OrdinaryDiffEq"]; mode = Pkg.PKGMODE_MANIFEST)

In [ ]:
VERSION # the Julia version

In [ ]:
using CairoMakie
update_theme!(resolution = (500, 300), fontsize = 16)

In [ ]:
# Let's read the overview of Attractors.jl
using DynamicalSystems
@doc Attractors

# Finding attractors and their basins

The bulk of the work in Attractors.jl is done by subtypes of `AttractorMapper`. Each subtype references a `DynamicalSystem`, and encapsulates an algorithm that allows to map initial conditions of the dynamical system into attractors. 
These `AttractorMapper`s compose an _extendable interface_.

The concrete subtypes that we'll cover here are:

1. `AttractorsViaRecurrences` 
2. `AttractorsViaFeaturizing`

## `AttractorsViaRecurrences`
Let's start with the recurrences algorithm. It implements the algorithm described in [Datseris & Wagemakers (2022), _Effortless estimation of basins of attraction_](https://doi.org/10.1063/5.0076568). Understanding of the algorithm is useful for deciding the keyword arguments of `AttractorsViaRecurrences`, however, explaining this algorithm in detail is not a part of this lecture due to time constraints! Worry not however, the exercises have been designed to work well with the default keywords!

The very short description is that, for "simple" dynamical systems like `CoupledODEs` or `DeterministicIteratedMap`, the Poincare recurrence theorem applies: it says that as the system state evolves on an attractor, it is guaranteed to visit a state space region it has already visited before. This is called a **recurrence**. The `AttractorsViaRecurrences` utilizes this fact to find attractors by tesselating a finite region of the state space into finite-sized cells, and coungint cells that have re-visited.

Here's a visual explanation:

<video width="50%" height="auto" controls autoplay loop>
<source src="https://raw.githubusercontent.com/JuliaDynamics/JuliaDynamics/master/videos/attractors/recurrence_algorithm.mp4?raw=true" type="video/mp4">
</video>

The only thing the algorithm needs is:

1. A dynamical system instance
2. A grid that both decides the state space region of interest as well as defines its tesselation

Remaining options are meta-parameters that tune the system behavior. Most cases work fine with default parameters so we won't be spending too much time on that...

We'll use `AttractorsViaRecurrences` to find attractors of the following dynamical system:
$$
\begin{aligned}
\dot{x} &= \sin(y) - bx\\
\dot{y} &= \sin(z) - by\\
\dot{z} &= \sin(x) - bz
\end{aligned}
$$
which is also called ["Thomas' cyclical system"](https://en.wikipedia.org/wiki/Thomas%27_cyclically_symmetric_attractor). Due to the symmetry of the equations of motion, any attractor has three symmetric versions (by cycling over the x, y, z coordinates). So this is a multistable dynamical system! 

For `b = 0.16` the system has three (symmetric) attracting limit cycles that can be found in the state space box spanning -3 to 3 in each dimension (we know this a-priori for this example).

Let's define the dynamical system:

In [ ]:
using DynamicalSystems

function thomas_rule(u, p, t)
    x,y,z = u
    b = p[1]
    xdot = sin(y) - b*x
    ydot = sin(z) - b*y
    zdot = sin(x) - b*z
    return SVector(xdot, ydot, zdot)
end

thomas = CoupledODEs(thomas_rule, ones(3), [0.16])

Next, we need to define the state space grid. This is as easy as defining a range for each of the dimensions and putting them in a tuple:

In [ ]:
xg = yg = zg = range(-6.0, 6.0; length = 101)

grid = (xg, yg, zg)

Now we use these two pieces of information to initialize the `AttractorMapper` we want:

In [ ]:
mapper = AttractorsViaRecurrences(thomas, grid)

The main job of a mapper is to tell you to which attractor an initial condition `u0` converges to via the syntax `mapper(u0)`. For example,

In [ ]:
mapper([0.5, 2.0, 1.0])

In [ ]:
mapper([0.5, 2.0, -1.0])

In [ ]:
mapper([0.5, 2.5, -1.0])

Attractors are labelled with the positive integers.

With this mapper we could now compute the **basins of attraction** of `thomas` by evolving an initial condition for each point in the state space and coloring it according to the attractor it converges to.  However, this is computationally tractable only for very low dimensional dynamical systems.

Therefore, the main way to use an `AttractorMapper` is with the `basins_fractions` function. You give it the mapper, and some initial conditions, and `basins_fractions` will map these initial conditions to attractors, collect their labels (i.e., which attractor they end up to), and return the fractions of basins of attraction.

Let's create a function that generates initial conditions on the fly using the existing `statespace_sampler` library function

In [ ]:
sampler, = statespace_sampler(grid)
sampler()

In [ ]:
sampler()

Then we give the mapper and the sampler to `basins_fractions`:

In [ ]:
@time fractions = basins_fractions(
    mapper, sampler;
    show_progress = false,
    N = 100_000, # how many random samples to use
)
fractions

As you can see, the function collected all unique attractor labels and created a dictionary mapping the labels to the fractions of initial conditions that went to the given attractor. This is called the state space fraction of the basins of attraction, or **the basins fractions** for sort.

We can extract the attractors as a dictionary mapping the labels to attractors (`StateSpaceSet`s)

In [ ]:
attractors = extract_attractors(mapper)

Let's visualize the attractors found!

In [ ]:
fig = Figure()
ax = Axis3(fig[1,1])
for (k, A) in attractors
    # Scatter the found points on the attractor by the recurrences algorithm
    scatter!(ax, vec(A); markersize = 5)
    # Also evolve an i.c. on the attractor and line plot it
    X, t = trajectory(thomas, 50.0, A[1])
    xt, yt, zt = columns(X)
    lines!(ax, xt, yt, zt)
end
fig

## `AttractorsViaFeaturizing`

This is an alternative way to find "attractors", which works as follows:

1. Initial conditions are evolved to generate trajectories.
2. Trajectories are mapped into "features": descriptive numbers of the trajectories that hopefully characterize different attractors differently. E.g., mean and std. of the first two  variables for a four-dimensional system.
3. Features are grouped using one of many choices, such as a clustering algorithm.
4. The created groups then correspond to unique "attractors".



Often it is not obvious what features would be good to separate the attractors, which is why we recommend the recurrences method when we know very little about the system.

In any case, for the Thomas cyclical example we have estimated in advance a featurizer that works well:

In [ ]:
using Statistics: mean, std

function featurizer(A, t) # take in trajectory and time vector
    x, y, z = columns(A)
    mx = mean(x)
    sx = std(y)
    cz = std(y .- z)
    return SVector(mx, sx, cz) # return feature, preferably as static vector
end

We also choose how to group features together. In this example, we will use a black box clustering algorithm called DBSCAN.

In [ ]:
grouping_config = GroupViaClustering();

We now put everything together in `AttractorsViaFeaturizing` and get the fractions of the basins of attraction:

In [ ]:
mapperf = AttractorsViaFeaturizing(
    thomas, featurizer, grouping_config;
    Ttr = 100.0, T = 100.0, Δt = 0.5
)

fs = basins_fractions(
    mapperf, sampler;
    show_progress = false
)

To illustrate what the algorithm did, we'll write some code that is internally used by `AttractorsViaFeaturizing`:

In [ ]:
N = 1000
all_features = StateSpaceSet{3, Float64}()
for _ in 1:N
    u0 = sampler()
    A, t = trajectory(thomas, 100.0, u0; Ttr = 100.0, Δt = 0.5)
    feature = featurizer(A, t)
    push!(all_features, feature)
end

labels = group_features(all_features, grouping_config)

c1, c2 = columns(all_features)
scatter(c1, c2; color = labels, axis = (xlabel = "feature 1", ylabel = "feature 2", title = "color = group (attractor)"))

# Continuation of attractors and their basins fractions

Continuation in the context of Attractors.jl is done using the `continuation` function, so let's read its documentation string:

In [ ]:
@doc continuation

`continuation` provides several methods for finding and continuing attractors, and also defines an extendable interface for adding future methods. Here we will focus on the Recurrences Find and Match (RAFM). 

Applying this method to the cyclical Thomas system we analyzed in the previous section is not interesting, because due to system symmetry the basins fractions are constantly 1/3 each. Instead, let's try to find, and continue, the attractors and the basins of attraction of the model from:

> Synthetic multistability in mammalian cells, Zhu et al., Science 375 (2022), DOI: 10.1126/science.abg9765

which is a modeing attempt at cells that may differentiate their behavior based on ongoing biological processes.

$$
\dot{A_i} = \alpha + \beta \frac{ B_i^n }{ 1 + B_i^n } - A_i, \quad i = 1, \ldots, N, 
$$

with

$$
B_i = \frac{2A_i^2}{ K_d + 4\sum A_i  + \sqrt{ K_d^2 + 8\sum A_i + K_d } }
$$

Here we will use $N=3$, which makes a 3-dimensional continuous time system. The starting parameters will be `α=4, β=20, n=1.5, Kd=80.0`. We are interested in the behavior of the model versions the `Kd` parameter. 

For the recurrences algorithm we do not need much accuracy for this particular system. The state space tesselation will therefore be from 0 to 100 for each dimension, divided into 100 cells.

In [ ]:
# Define dynamical system
using OrdinaryDiffEq: Vern9
function cell_differentiation(N = 3, u0 = rand(N); α=4, β=20, n=1.5, Kd=1.0)
    p = [Kd, α, β, n]
    diffeq = (alg = Vern9(), reltol = 1e-9, abstol = 1e-9)
    ds = CoupledODEs(cell_differentiation_rule!, u0, p; diffeq)
    return ds
end
function cell_differentiation_rule!(du, u, p, t)
    Kd, α, β, n = p
    sum_u = sum(u)
    @inbounds for i ∈ eachindex(du)
        C = (2*u[i]^2) / (Kd + 4*sum_u + sqrt( Kd^2 + 8*sum_u*Kd )  )
        du[i] = α + (β*C^n)/(1+C^n) - u[i]
    end
    return nothing
end

ds = cell_differentiation(3)

We then create an `AttractorsViaRecurrences` instance

In [ ]:
grid_cell = ntuple(i -> range(0, 100; length=101), 3)

mapper = AttractorsViaRecurrences(
    ds, grid_cell; maximum_iterations = Int(1e9)
)

And now the only thing we need to do to continue these attractors across a parameter range is to initialize a `RecurrencesFindAndMatch` instance with the `mapper`, and give it to the `continuation` function!

In [ ]:
sampler, = statespace_sampler(grid_cell)

In [ ]:
rsc = RecurrencesFindAndMatch(mapper; threshold = Inf)

pidx = 1 # index of parameter to change (here Kd)
prange = range(1e-2, 1e2; length = 101) # parameter range to scan

@time fractions_curves, attractors_info = continuation(
    rsc, prange, pidx, sampler;
    show_progress = false, samples_per_parameter = 1000
)
fractions_curves

Let's now plot the results we found using one of the provided plotting functions from Attractors.jl:

In [ ]:
# Decide how to plot attractors: go from attractor to real number
attractor_to_real = A -> maximum(x[1] for x in A)

plot_basins_attractors_curves(fractions_curves, attractors_info, attractor_to_real, prange)

## Matching attractors

Advanced. Do we have time...?

## Sidenote: local bifurcation & continuation

In our opinion, the approach of Attractors.jl supercedes the decades-old approach  used in nonlinear dynamics that is the local bifurcation & continuation method on the basis of Newton's algorithm (and more advanced variants). That is, we believe that a practitioner is better off using Attractors.jl most of the time, and only turning to the old approach when e.g., interested in unstable sets, or an automated analysis of structural change of the Jacobian eigenvalues (i.e., a local bifurcation). Traditional continuation analysis in Julia can be done with BifurcationKit.jl, which is not taught as part of these lectures.

# Other measures of global stability

By default, `continuation` continues the attractors and their basin fractions. The basin fractions is the most basic measure of **global stability** (also known as **non-local stability** or **resilience**). But there are other ways to quantify global stability, some of which are also available out of the box.

Another typical measure of global stability is the **minimal fatal shock** (MFS), that is, the perturbation of least mangitude we need to do on a state to push the state into a basin of another attractor. In Attractors.jl the function `minimal_fatal_shock` computes this quantity for _any_ dynamical system. This is a huge improvement over the paper that originally introduced the concept, as there it was only possible to compute for a very specific class of dynamical systems.

The way `minimal_fatal_shock` works is very simple. Give in a dynamical system, a state `u0` and the function will return the MFS. Let's use this at the cell differention model. We first find all its attractors, and then we find the MFS correpsonding to each attractor.

In [ ]:
cell = cell_differentiation(3)
set_parameter!(ds, 1, 80.0)
mapper_cell = AttractorsViaRecurrences(ds, grid_cell)
sampler_cell, = statespace_sampler(grid_cell, 12345)
fractions = basins_fractions(mapper_cell, sampler_cell; show_progress = false)
attractors = extract_attractors(mapper_cell)

Obtain the MFS for each attractor

In [ ]:
search_area = (0.0, 100.0)
mfs_algo = MFSBruteForce(1000, 2000, 0.995) # B.B. optimization doesn't work here due to NaNs
@time shocks = Dict(
    i => minimal_fatal_shock(mapper_cell, A[1], search_area, mfs_algo)
    for (i, A) in attractors
)

In [ ]:
using LinearAlgebra: norm
shock_magnitudes = Dict(i => norm(s) for (i, s) in shocks)

# Exercises

## Basins of attraction of a fast-slow multistable model

The goal of this exercise is to demonstrate that `AttractorsViaRecurrences` is not magic, and has its own limitations. 

Consider the following (modified) form of the Rozenweig-MacArthur continuous time model for predator-prey dynamics:
$$

\begin{align}
\frac{dN}{dt} & = rN\left(1-\frac{c}{r}N\right)\left(\frac{N-\mu}{N+\nu}\right) - A \\
\frac{dP}{dt} & = \chi A - \delta P \\
A & = \frac{\alpha N P}{\beta + N} 
\end{align}
$$

with initial parameters `p = [2.0, 0.19, 0.03, 0.003, 800, 1.5, 0.004, 2.2] = r, c, μ, ν, α, β, χ, δ`. 

After creating the dynamical system instance, create a `AttractorViaRecurrences` instance, using the following grid. Set your own choice for `consecutive_recurrences`, which is the keyword that decides how many recurrences we should count before declaring convergence to an attractor.

```julia
density = 101
xg = range(-0.1, 20; length = density)
yg = range(-0.0001, 0.03; length = density)
grid = (xg, yg)

mapper = AttractorsViaRecurrences(
    ds, grid;
    consecutive_recurrences = YOUR_CHOICE, Δt = 0.01,
)
```

Then, use the function `basins_of_attraction` to estimate the basins of attraction.
For estimating the basins, avoid initial conditions with ≤ 0 values, i.e., use 
```julia
basinsgrid = (xg[2:end], yg[2:end])
basins, attractors = basins_of_attraction(mapper, basinsgrid)
```
You may visualize these results using 
```julia
fig = heatmap_basins_attractors(basinsgrid, basins, attractors)
```

Repeat this calculation while varying `consecutive_recurrences`. You will notice that for low values of `consecutive_recurrences` the computation fails with an informative error. Can you understand why? Use the `interactive_evolution` application of DynamicalSystems.jl and evolve some initial conditions to help you comprehend the problem.

## Continuation of basins fractions and Hopf bifurcation identification

Continue from the previous exercise. To ensure accurate attractor identification, use the following keywords for `AttractorsViaRecurrences`
```julia
consecutive_recurrences = 5000, Δt = 0.1,
force_non_adaptive = true, Ttr = 100.0,
```
Use the `continuation` function to continue attractors and their fractions across the parameter range `r = 1` to `2`. Use the `statespace_sampler` function to generate random initial conditions in the `basinsgrid` you defined above. Plot the fractions of basins of attraction versus the parameter range, and the attractor value, using the following function to map attractors to real numbers:
```julia
using Statistics: mean
attractor_to_real = A -> mean(x[1] for x in A)
```

The solution of the exercise should be that you have only two attractors and the fractions of their basins are approximately 50% all the time.

One of the two attractors is a fixed point that becomes a limit cycle, i.e., it undergoes a Hopf bifurcation. Find the parameter value where this occurs by analyzing the attractors corresponding to the ID of the "fixed point -> limit cycle" attractor. Identify the bifurcation point by checking when then `length` of the attractor becomes larger than 1. (this means, the attractor spans more than 1 cells in the tesselated state space)

## Minimal fatal shock for generic attractors

The minimal fatal shock (MFS) concept was originally defined for fixed point attractors. For limit cycles or chaotic attractors the concept is less useful as there is no specific state on the attractor from which it is the most relevant to obtain the MFS. In such cases one would have to obtain MFSs for many points on the attractor. This exercise is about doing exactly that!

Continue from the above definition of the Thomas cyclical system with parameter `b = 0.16`. Obtain any of the three attractors of the system by doing `id = mapper([1, 2, 3.0])`, and then `A = extract_attractors(mapper)[id]`. Calculate the MFS for various states on the attractor `A` and collect all these MFSs in a vector. What is the average shock magnitude? Plot a scatter plot of the shock mangitude on the attractor (i.e., scatterplot the points of `A` while assigning as color the mangitude of the corresponding MFS).
